# Imputation of missing values

From this guide: https://scikit-learn.org/stable/modules/impute.html

## Overview

For various reasons, many real world datasets contain missing values, often encoded as blanks, NaNs or other placeholders. Such datasets however are incompatible with scikit-learn estimators which assume that all values in an array are numerical, and that all have and hold *meaning*. 

A basic strategy to use incomplete datasets is *to discard entire rows and/or columns* containing missing values. However, this comes at the price of losing data which may be valuable (even though incomplete). 

A better strategy is to **impute** the missing values, i.e., to infer them from the known part of the data. 

## [Univariate vs. Multivariate Imputation](https://scikit-learn.org/stable/modules/impute.html#univariate-vs-multivariate-imputation)

One type of imputation algorithm is **univariate**: 
* imputes values in the i-th feature dimension using only non-missing values in that feature dimension (e.g. `impute.SimpleImputer`). 

By contrast, **multivariate** imputation algorithms:
* use the *entire set of available feature dimensions* to estimate the missing values (e.g. `impute.IterativeImputer`).

## [Univariate feature imputation](https://scikit-learn.org/stable/modules/impute.html#univariate-feature-imputation)

The `SimpleImputer` class provides basic strategies for imputing missing values. 

Missing values can be imputed with: 
* a provided constant value, or 
* using the statistics (`mean`, `median` or `most frequent`) 

of each column in which the missing values are located. 

This class also allows for *different missing values encodings*.

The following snippet demonstrates how to replace missing values, encoded as `np.nan`, using the *mean value* of the columns (axis 0) that contain the missing values:

In [2]:
from IPython.display import display
import numpy as np
from sklearn.impute import SimpleImputer

imp = SimpleImputer(
    missing_values=np.nan, 
    strategy='mean'
)

imp.fit([[1, 2], [np.nan, 3], [7, 6]])

X = np.array(
    [
        [np.nan, 2], 
        [6, np.nan], 
        [7, 6]
    ]
)

print(X)
print(imp.transform(X))

[[nan  2.]
 [ 6. nan]
 [ 7.  6.]]
[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


The `SimpleImputer` class also supports **sparse matrices**:

In [5]:
import scipy.sparse as sp

# CSC sparse matrix:
X = sp.csc_matrix(
    [
        [1, 2], 
        [0, -1], 
        [8, 4]
    ]
)
print("X:\n", X)

imp = SimpleImputer(
    missing_values=-1, 
    strategy='mean'
)

imp.fit(X)

X_test = sp.csc_matrix(
    [
        [-1, 2], 
        [6, -1], 
        [7, 6]
    ]
)

print("X_test before:\n", X_test)
X_test_after = imp.transform(X_test)
print("X_test after:\n", X_test_after)

X:
   (0, 0)	1
  (2, 0)	8
  (0, 1)	2
  (1, 1)	-1
  (2, 1)	4
X_test before:
   (0, 0)	-1
  (1, 0)	6
  (2, 0)	7
  (0, 1)	2
  (1, 1)	-1
  (2, 1)	6
X_test after:
   (0, 0)	3.0
  (1, 0)	6.0
  (2, 0)	7.0
  (0, 1)	2.0
  (1, 1)	3.0
  (2, 1)	6.0


Note that this format is not meant to be used to implicitly store missing values in the matrix because it would densify it at transform time. Missing values encoded by 0 must be used with dense input.

The `SimpleImputer` class also supports categorical data represented as string values or [pandas categoricals](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) when using the `'most_frequent'` or `'constant'` strategy:

In [6]:
import pandas as pd
df = pd.DataFrame(
    [
        ["a", "x"],
        [np.nan, "y"],
        ["a", np.nan],
        ["b", "y"]
    ], 
    dtype="category"  # <-- NOTE.
)
display(df)

,0,1
0,a,x
1,NaN,y
2,a,NaN
3,b,y


In [7]:
imp = SimpleImputer(strategy="most_frequent")

imp.fit_transform(df)

array([['a', 'x'],
       ['a', 'y'],
       ['a', 'y'],
       ['b', 'y']], dtype=object)

## [Multivariate feature imputation](https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation)

A more sophisticated approach is to use the `IterativeImputer` class, which models each feature with missing values *as a function of other features*, and uses that estimate for imputation. 

It does so in an *iterated round-robin fashion*: 
* at each step, a feature column is designated as output `y` and the other feature columns are treated as inputs `X`. 
* A *regressor is fit* on `(X, y)` for known `y`. 
* Then, the regressor is used to predict the missing values of y. 

This is done for each feature in an iterative fashion, and then is repeated for `max_iter` imputation rounds. *The results of the final imputation round are returned.*

---

**⏱️⚙️ Note (Experimental as of version `0.24.2`)**

This estimator is still **experimental** for now: default parameters or details of behaviour might change without any deprecation cycle. Resolving the following issues would help stabilize `IterativeImputer`: convergence criteria ([#14338](https://github.com/scikit-learn/scikit-learn/issues/14338)), default estimators ([#13286](https://github.com/scikit-learn/scikit-learn/issues/13286)), and use of random state ([#15611](https://github.com/scikit-learn/scikit-learn/issues/15611)). To use it, you need to explicitly import `enable_iterative_imputer`.

---

In [10]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # NOTE!
from sklearn.impute import IterativeImputer

In [11]:
imp = IterativeImputer(
    max_iter=10, 
    random_state=0
)

imp.fit(
    [
        [1, 2], 
        [3, 6], 
        [4, 8], 
        [np.nan, 3], 
        [7, np.nan]
    ]
)

IterativeImputer(random_state=0)

In [12]:
X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first
print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


Both `SimpleImputer` and `IterativeImputer` can be used in a `Pipeline` as a way to build a *composite estimator* that supports imputation. 

**See example:** 📚 [Imputing missing values before building an estimator](https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py).

### Flexibility of IterativeImputer

There are many well-established imputation packages in the `R` data science ecosystem: `Amelia`, `mi`, `mice`, `missForest`, etc. 

`missForest` is popular, and turns out to be *a particular instance of different sequential imputation algorithms* that can all be implemented with `IterativeImputer` by passing in different regressors to be used for predicting missing feature values. In the case of `missForest`, this regressor is a `Random Forest`. 

**See example:** 📚🎓 [Imputing missing values with variants of IterativeImputer](https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py).

### Multiple vs. Single Imputation

ℹ️ In the **statistics community**, it is common practice to *perform multiple imputations*, generating, for example, `m` separate imputations for a single feature matrix. 

Each of these `m` imputations is then *put through the subsequent analysis pipeline* (e.g. feature engineering, clustering, regression, classification). 

The `m` final analysis results (e.g. held-out validation errors) allow the data scientist to obtain understanding of how analytic results may differ as a consequence of the inherent uncertainty caused by the missing values. 

**The above practice is called multiple imputation.**

💡🎓 Our implementation of `IterativeImputer` was inspired by the `R MICE package` (*Multivariate Imputation by Chained Equations*) `Stef van Buuren, Karin Groothuis-Oudshoorn (2011). "mice: Multivariate Imputation by Chained Equations in R". Journal of Statistical Software 45: 1-67.`, **but differs from it by returning a single imputation instead of multiple imputations**. However, `IterativeImputer` can also be used for *multiple imputations* by applying it repeatedly to the same dataset with different random seeds when `sample_posterior=True`. See `Roderick J A Little and Donald B Rubin (1986). “Statistical Analysis with Missing Data”. John Wiley & Sons, Inc., New York, NY, USA.`, chapter 4 for more discussion on multiple vs. single imputations.

## [Nearest neighbors imputation](https://scikit-learn.org/stable/modules/impute.html#nearest-neighbors-imputation)

The `KNNImputer` class provides imputation for filling in missing values using the k-Nearest Neighbors approach. 
* By default, a euclidean distance metric that supports missing values, `nan_euclidean_distances`, is used to find the nearest neighbors.
* Each missing feature is imputed using values from `n_neighbors` nearest neighbors that have a value for the feature.
* The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor.
* If a sample has more than one feature missing, then the neighbors for that sample can be different depending on the particular feature being imputed.
* When the number of available neighbors is less than `n_neighbors` and there are no defined distances to the training set, the training set average for that feature is used during imputation.
* If there is at least one neighbor with a defined distance, the weighted or unweighted average of the remaining neighbors will be used during imputation.
* If a feature is always missing in training, it is removed during transform (❗).

The following snippet demonstrates how to replace missing values, encoded as `np.nan`, using the mean feature value of the two nearest neighbors of samples with missing values:

In [13]:
import numpy as np
from sklearn.impute import KNNImputer

nan = np.nan
X = [
    [1,   2,   nan], 
    [3,   4,   3], 
    [nan, 6,   5], 
    [8,   8,   7]
]

imputer = KNNImputer(
    n_neighbors=2, 
    weights="uniform"
)

imputer.fit_transform(X)

array([[1. , 2. , 4. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])

## [Marking imputed values](https://scikit-learn.org/stable/modules/impute.html#marking-imputed-values)

The `MissingIndicator` transformer is useful to transform a dataset into corresponding **binary matrix indicating the presence of missing values** in the dataset. 

This transformation is useful in conjunction with imputation. When using imputation, preserving the information about which values had been missing can be informative. 

> Note that both the `SimpleImputer` and `IterativeImputer` have the boolean parameter `add_indicator` (`False` by default) which when set to `True` provides a convenient way of *stacking the output of the `MissingIndicator` transformer with the output of the imputer*.

`NaN` is usually used as the placeholder for missing values. **However, it enforces the data type to be `float`**. The parameter `missing_values` allows to specify other placeholder such as integer. In the following example, we will use `-1` as missing values:

In [14]:
from sklearn.impute import MissingIndicator
X = np.array(
    [
        [-1, -1, 1, 3],
        [4, -1, 0, -1],
        [8, -1, 1, 0]
    ]
)

indicator = MissingIndicator(missing_values=-1)  # <-- See this: integer `-1` interpreted as missing indicator.

mask_missing_values_only = indicator.fit_transform(X)
mask_missing_values_only

array([[ True,  True, False],
       [False,  True,  True],
       [False,  True, False]])

The `features` parameter is used to choose the features for which the mask is constructed. By default, it is `'missing-only'` which returns the imputer mask of the *features containing missing values at fit time*:

In [16]:
indicator.features_
# Note feature of index 2 is missing here, it didn't have any missing indicators at fit time (here, -1s)

array([0, 1, 3])

The `features` parameter can be set to `'all'` to return all features whether or not they contain missing values:

In [18]:
indicator = MissingIndicator(
    missing_values=-1, 
    features="all"  # <-- This.
)

mask_all = indicator.fit_transform(X)
mask_all

array([[ True,  True, False, False],
       [False,  True, False,  True],
       [False,  True, False, False]])

In [19]:
indicator.features_

array([0, 1, 2, 3])

⚠️ When using the `MissingIndicator` in a `Pipeline`, be sure to use the 
* `FeatureUnion` or 
* `ColumnTransformer` 

to add the indicator features to the regular features.

First we obtain the `iris` dataset, and add some missing values to it.

In [20]:
from sklearn.datasets import load_iris
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier

In [21]:
X, y = load_iris(return_X_y=True)

In [22]:
# Set some values to NaN.
mask = np.random.randint(0, 2, size=X.shape).astype(bool)
X[mask] = np.nan

In [23]:
X_train, X_test, y_train, _ = train_test_split(
    X, 
    y, 
    test_size=100,
    random_state=0
)

Now we create a `FeatureUnion`. 

All features will be imputed using `SimpleImputer`, in order to enable classifiers to work with this data. 

Additionally, it adds the indicator variables from `MissingIndicator`.

In [24]:
transformer = FeatureUnion(
    transformer_list=[
        ('features', SimpleImputer(strategy='mean')),
        ('indicators', MissingIndicator())
    ]
)
display(transformer)

transformer = transformer.fit(X_train, y_train)
results = transformer.transform(X_test)
results.shape

FeatureUnion(transformer_list=[('features', SimpleImputer()),
                               ('indicators', MissingIndicator())])

(100, 8)

Of course, we cannot use the transformer to make any predictions. We should wrap this in a `Pipeline` with a classifier (e.g., a `DecisionTreeClassifier`) to be able to make predictions.

In [25]:
clf = make_pipeline(transformer, DecisionTreeClassifier())  # Using convenience function `make_pipeline()`
clf = clf.fit(X_train, y_train)
results = clf.predict(X_test)
results.shape

(100,)